# Age Cross-Prediction

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

## Get Data

In [23]:
from common.data import get_data
from common.paths import ADHD

X, Y, demographics, population = get_data(wisc_level=5, label_path=ADHD)
ages = demographics['Age']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (373, 34716) | Y: 6 | Age: (373,) | Population: adhd


In [15]:
from common.binning import bin_data
from common.cross_prediction import get_group_cv_splits, get_group_order
from common.paths import CROSS_PRED_RESULTS, CROSS_PRED_PSCORES
from common.results import CVResult, save_results, save_perm_score
from common.scoring import (unimetric_scorer, 
                            cross_prediction_permutation_test_score, 
                            N_PERM, SCORING, RKF_10_10)
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [24]:
selected_target = "WISC_VCI"
y = Y[selected_target]

print(f'{selected_target}: {y.shape}')

WISC_VCI: (373,)


In [25]:
from common.binning import bin_data

X_bins, y_bins, bin_labels = bin_data(X, y, ages, False, 3)

for age_bin, bin_label in zip(X_bins, bin_labels):
    print(f'{bin_label}: {age_bin.shape}')

Bin 1: (114, 34716)
Bin 2: (147, 34716)
Bin 3: (112, 34716)


In [26]:
bin_2_subsample_indices = np.random.choice(147, 113, replace=False)
X_bins[1] = X_bins[1][bin_2_subsample_indices]
y_bins[1] = y_bins[1][bin_2_subsample_indices]
bin_labels[1] = 'Bin 2 Equal'

print(X_bins[1].shape, y_bins[1].shape)

(113, 34716) (113,)


In [27]:
bins = [(X_bin, y_bin) for X_bin, y_bin in zip(X_bins, y_bins)]
bins_cv = get_group_cv_splits(bins, RKF_10_10)

print(f'Bin 1: {bins[0][0].shape} | Bin 2: {bins[1][0].shape} | Bin 3: {bins[2][0].shape}')
print(f'bin_1_cv: {len(bins_cv[0])} | bin_2_cv: {len(bins_cv[1])} | bin_3_cv: {len(bins_cv[2])}')

Bin 1: (114, 34716) | Bin 2: (113, 34716) | Bin 3: (112, 34716)
bin_1_cv: 100 | bin_2_cv: 100 | bin_3_cv: 100


### Run permutation-test (train bin, test bin)

In [ ]:
%%time

bin_alphas = [1, 6901, 7101]  # From previous results

bin_order, cv_order, label_order = get_group_order(bins, bins_cv, bin_labels)
results = []
# perm_scores = []

for bin_alpha, bins, bins_cv, labels in zip(bin_alphas, bin_order, cv_order, label_order):
    train_bin, test_bin_one, test_bin_two = bins[0], bins[1], bins[2]
    train_bin_cv, test_bin_one_cv, test_bin_two_cv = bins_cv[0], bins_cv[1], bins_cv[2]
    
    pipe = make_pipeline(StandardScaler(), Ridge(alpha=bin_alpha))
    rs, perms, ps = cross_prediction_permutation_test_score(
        pipe, train_bin, test_bin_one, test_bin_two, 
        train_bin_cv, test_bin_one_cv, test_bin_two_cv, N_PERM, unimetric_scorer)
    
    train_group = labels[0]
    for r, p, test_group in zip(rs, ps, labels):
        results.append(
            CVResult('ridge', selected_target, train_group, test_group, r, p, population, N_PERM)
        )
#     perm_scores.append(perms)
    print(f'Train Group: {train_group}')
    save_perm_score(perms, f'ridge_{population}_{selected_target}_{train_group}_cross_prediction_perm_scores', CROSS_PRED_PSCORES)

results_df = pd.DataFrame([r.to_dict() for r in results])
display(results_df.round(4))
filename = 'ridge_pts_age_cross_prediction_bin_two_equal_samples'
results_fp = save_results(results_df, filename, CROSS_PRED_RESULTS)
print('Results saved to:', results_fp)

Train Group: Bin 1
Train Group: Bin 2 Equal
